In [1227]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys

In [1228]:
#PARAMS
p = 31
Fp = GF(p)
n = 256
k = 52
r = n-k
t = n
z = 3
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])

In [1229]:
#FUNZIONI DI UTILITA

def generate_random_seed():
    set_random_seed()
    return initial_seed()

def set_seed(seed):
    set_random_seed(seed)
    return initial_seed()

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

def convert_int_to_hash_string(input):
    return input.to_bytes((input.bit_length() + 7) // 8, 'big').hex()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2
#t=8
# #w = 5

#0,2,4,5,7

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    set_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    set_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed):
    eta = create_random_vector(Fz, n, seed)
    e = vector(Fp, n)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def create_vector_over_Fp_star(Fp_star, t, seed):
    set_random_seed(seed)
    return [Fp_star.random_element() for i in range(t)]

def componentwise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

In [1230]:
#SEED TREE
class SeedTree:
    def __init__(self, depth, salt, seed_m):
        if(depth>0):
            self.seed_m = seed_m
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree(self.seed_m, 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def get_seed_path(self, list_nodes):
        if self.full:
            self.seed_path={}
            self.__get_seed_path(self.depth-1, list_nodes)
            return self.seed_path
        else:
            raise Exception('Albero non creato o foglia non valida')

#da finire
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth<=self.depth):
            list_children = self.__create_new_level(list_nodes, depth)
            self.tree.append(list_children)
            self.__make_tree(list_children, depth+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents, '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    #provvisorio
    def __get_seed_path(self, depth, list_index):
        if len(list_index)>0 and depth>0:
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))#prendo gli indici dei nodi padri eliminando i duplicati
            list_sub_root = list(filter(lambda x: ((x % 2 == 0 and x + 1 not in list_parent) or (x % 2 != 0 and x - 1 not in list_parent)), list_parent))
            list_new_children = list(filter(lambda x:  x not in list_sub_root, list_parent))
            self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_new_children))
            self.__get_seed_path(depth-1, list_new_children)
        

    #da finire
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()
 

In [1231]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2)
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full):
            self.tree = []
            self.tree.append(leaves)
            self.__make_tree(leaves, self.depth-1)
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            list_leaves = list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves))
            list_indexes = self.__get_indexes(self.depth, list_leaves, [])
            self.__get_merkle_proof(list_indexes)
            return self.merkle_proof
        else:
            raise Exception('Albero non creato o foglia non valida')

    #VEDREMO
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth>=0):
            list_parent = self.__create_new_level(list_nodes)
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1)

    def __create_new_level(self, child_nodes):
        parents = []
        for i in range(0, len(child_nodes), 2):
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1])
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __get_indexes(self, depth, list_index, list_proof):
        if(depth > 0):
            list_proof.insert(0, list_index)
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))
            return self.__get_indexes(depth-1, list_parent, list_proof)
        else:
            return list_proof


    def __get_merkle_proof(self, list_merkle_proofs):
        self.merkle_proof = {}    
        for i in range(len(list_merkle_proofs)): 
            current_proof = list_merkle_proofs[i] #controllo livello i esimo
            if i < len(list_merkle_proofs) - 1:
                next_proof = list_merkle_proofs[i+1]
                current_proof = list(filter(lambda x: x*2 not in next_proof and (x*2)+1 not in next_proof, current_proof))
            if not len(current_proof)==0:
                self.merkle_proof[i+1] = dict(map(lambda x: (x, self.tree[i+1][x]), current_proof))

    #da finire
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()

In [1232]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_random_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed_e)
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [1233]:

class Signer:
    def __init__(self, seed_sk, seed_pk, s):
        self.seed_sk = seed_sk
        self.seed_pk = seed_pk
        self.seed_e = None
        self.salt = None
        self.V = None
        self.eta = None
        self.s = s
        self.seed_tree = None
        self.merkle_tree = None
        self.c0 = None
        self.c1 = None
        self.ch1 = None
        self.ch2 = None
        self.h = None
        
    
    def key_material_expansion(self):
        self.seed_e , self.seed_pk = expand_seed(self.seed_sk)
        self.V = create_random_matrix(Fp, k, r, self.seed_pk)
        self.eta = create_random_vector(Fz, n, self.seed_e)

    def compute_commitment(self):
        self.seed_m = generate_random_seed() 
        seed_salt = generate_random_seed()
        self.salt = hash_and_get_seed(seed_salt)
        self.seed_tree = SeedTree(lev_leaf, self.salt, self.seed_m)
        self.seed_tree.make_tree()
        self.seeds = self.seed_tree.get_leaves()
        cmt0 = [] 
        cmt1 = []
        for i in range(0, t):
            seed_ui, seed_ei = expand_seed(self.seeds[i])
            eta_i = create_random_vector(Fz, n, seed_ei)
            sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
            v = vector(Fp,n) #v∈ Fp^n
            g = get_generator(p, z, Fp)
            for j in range(0, n-1):
                v[j] = (g**sigma_i[j])  
            ui = create_random_vector(Fp, n, seed_ui)
            u = componentwise_multiply(v, ui)
            st = u[0:r] + u[r:n]*V
            commit0 = hexhash( str(st) + str(sigma_i) + str(self.salt) + str(i) )
            commit1 = hexhash( str(self.seeds[i]) + str(self.salt) + str(i) )
            cmt0.append(commit0) 
            cmt1.append(commit1)
        self.merkle_tree = MerkleTree(t)
        self.merkle_tree.make_tree(cmt0)
        self.c0 = self.merkle_tree.get_root()
        input_c1 = "".join(map(str, cmt1))
        self.c1 = hexhash(input_c1)

    def generate_first_challenge(self, msg):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch1 = self.c0+self.c1+msg+hash_salt
        seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, seed_ch1)

    def generate_first_response(self):
        list_hash_yi=[]
        for seed_i, ch_i_1 in zip(self.seeds, self.ch1):
            seed_ui, seed_ei = expand_seed(seed_i)
            g = get_generator(p, z, Fp)
            ei = create_vEn(g, n, Fz, Fp, seed_ei)
            ui = create_random_vector(Fp, n, seed_ui)
            yi = ui+(ch_i_1)*ei
            hi = hexhash(str(yi))
            list_hash_yi.append(hi)
        self.h = hexhash("".join(list_hash_yi))

    def generate_second_challenge(self, msg):#, seed_tree):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch2 = self.c0+self.c1+"".join(map(str, self.ch1))+self.h+msg+hash_salt
        seed_ch2 = hash_and_get_seed(input_ch2)
        self.ch2 = create_random_vector(GF(2), t, seed_ch2)

    def generate_second_response(self):
        index_1, index_0 = [], []
        [index_1.append(i) if x == 1 else index_0.append(i) for i, x in enumerate(self.ch2)]
        merkle_proofs = self.merkle_tree.get_proof(index_0)
        seed_path = self.seed_tree.get_seed_path(index_1)
        print(merkle_proofs)
        print(seed_path)
        #return merkle_proofs, seed_path

    def sign_msg(self, msg):
        self.key_material_expansion()
        self.compute_commitment()
        self.generate_first_challenge(msg)
        self.generate_first_response()
        self.generate_second_challenge(msg)
        self.generate_second_response()


In [1234]:
seed_sk, seed_pk, s = key_gen()
signer = Signer(seed_sk, seed_pk, s)
signer.sign_msg("buonasera")

{5: {6: '1592313ed3c86a45bdd23b55890170dfa46928d3d4f99d15cacf32e955cad012', 8: '833bbab90875a6b36a121d6df44a287e34bb8b4320b3340ddd37549ce5c1ee4d', 27: '4c264f3c7e5e0d2c03ff698c8f203f23a422cb80c83bb7385eb8e8560dd10921', 28: 'd75e94ae763fc742848baf888b10b403c081bfb7313afc0db4fe00c6ca3cd46f'}, 6: {5: '147d3d99aac9f28a7a69b9092c229a2d3ff81aff4086937436782dc3aaf8d0df', 11: '85549b1516cd72f85be472b12d2b5404ce4f130686bd5563bc1fc37b9f429b66', 20: 'eb835e679229b6590031022705e7a09d13237c1c57bdef3cd1e7bf80467c50c5', 28: '62079240a80341e85e86ee7c742473fe011b2b84ef6a4c7d41cd6001c53167e7', 38: 'c148837087cc4a5f8476e78a8526fb10551002a8b911083f52e1eef3a0ff60d8', 41: '309b90c8efe799e96e12b39446d339fb5a1ba2b3da16362c183bf3bf0a8f75c8', 47: 'f86b46f03f0790ab0a4336703c629eafa022f82a5f7fbc1f60e2079af7a3568e', 61: '9aec1c303a63acb38fc285f39c4f85b2687984f3b1b8558472fba5a6fd03257f'}, 7: {0: '09cca1b0dd5965a939f04e87cca89dde3079199ddec2cdca0351211b4d02bbfa', 2: 'facc6809b9274c61cdfd8f08227a43010524ce4c626f8a071

In [1235]:
print(merkle_proofs)


{5: {6: '10f472b886ea12cb05523635730083c61f12b11fbc0b81083e9bf3849f12aa62'}, 6: {1: 'ec148dfbde412f83eccdfae4ccbdb08569132cdb7eed22c4074f99096642e82a', 17: 'e5d39cedd392f47eaad4b5523b6741b2004c4faf517bec6567f58c245d93ae72', 19: 'dd64ba81a727a7b47824bc59d4dda33d99695e9647dd84d3377074bd852b9496', 25: 'bec86bd2397fc767acb85b1f63f144d38a7545558d1c554aab660ccd2307b335', 27: '332e447f3085c743bc14b9f42841d3c66057f32dcfa3d59b6a122e1d05fc6e31', 29: '103033df1dc108d0734df1d0adf6a2b8fdabd068903f36d973ad6cbae99f4af4', 31: 'f7f1e1a7e00ed8d3cc7d74a78e3c88080f9a37cd601c52a7f8a47fd578bed134', 32: '0f83533261cb74c5bc5fbc8761d367d8c0d6c09d1a35b82816b62dc36003ff21', 34: '025585be21578a5d5606d883537b14b1c86be465a26ca9e40a2fd56468522a4d', 38: '107acae8c6181ff6aef2b138e10353d641f2751c719e00a7578d0d8b7341cd60', 41: 'c572b720c124ade219477336809ae9b23d74bae32beeee61b315d1f69479afaa', 43: 'b774a10bfb673ba77620e122301d4b05e24cffee10da3b96841a68a5b14605ff', 44: '78883cc069d7c1c33336a0d856616d83626dbcf2f56ad8b59e9

In [1236]:
print(seed_path)

{7: {0: 164273963905989165588759198958074554932, 1: 3537925272254563838177049641685907146, 6: 147829883941797379651304561115680246401, 7: 19485149138262798889166087941727880750, 8: 335348552034713675247440212481859238158, 9: 50803162634428998540483647922448917438, 10: 36967965553328247353791684720846513189, 11: 260572871468166729018093204856697646807, 12: 106707829708547557328967017018140777660, 13: 333654742313892429930952580663293241204, 16: 265457715466165910453461736435149303053, 17: 249134447021296571764954957816152765055, 18: 193937241004152699143678915249664553854, 19: 110475487618567339544062274169046370005, 26: 160876448073003534987348509508921453364, 27: 334851472669510550835654465639198931116, 28: 1132686328025057673061993723926489748, 29: 128328386721307983800252627661632486802, 30: 4739141407238389781139511579735762768, 31: 87376045200292122887769016997974415602, 36: 293665884029809953249765863234953235925, 37: 175598435267282010750973891624328791976, 42: 31323626847189126